Import dataset


In [ ]:
!pip install mnist

In [ ]:
import numpy as np
import mnist

Data preparation


In [ ]:
#digunakan untuk reshape array gambar.
def reshapeImages(images):
    images = images.reshape(images.shape[0], -1)
    return images

#digunakan oleh reshapeImages untuk mereshape data train dan test dari dataset
def reshapedMnistData(train_images, train_labels, test_images, test_labels):
    train_images = reshapeImages(train_images)
    train_labels = reshapeImages(train_labels)
    test_images = reshapeImages(test_images)
    test_labels = reshapeImages(test_labels)
    return train_images, train_labels, test_images, test_labels

#fungsi ini memuat dataset mnist
def getMnistData(reshaped=True):
    train_images = mnist.train_images()
    train_labels = mnist.train_labels()
    test_images = mnist.test_images()
    test_labels = mnist.test_labels()
    if reshaped == True:
        return reshapedMnistData(train_images, train_labels, test_images, test_labels)
    else:
        return train_images, train_labels, test_images, test_labels


Conv

In [ ]:
class Conv(object):

#konstruktor kelas inisiasi inisiasi objek conv dengan jumlah filter
    def __init__(self, num_filters):
        self.num_filters = num_filters
        self.filters = np.random.randn(num_filters, 3, 3, ) / 9

#untuk mengiterasi melalui semua region 3x3 di dalam gambar.
#region ini digunakan untuk menghitung hasil konvolusi
    def iterate_regions(self, image):
        h, w = image.shape
        for i in range(h - 2):
            for j in range(w - 2):
                im_region = image[i:i+3, j:j+3]
                yield im_region, i, j

#melakukan proses konvolusi pada input.
#matriks input diiterasi melalui semua region 3x3 menggunakan iterate_region
    def forward(self, input):
        self.last_input = input
        h, w = input.shape
        output = np.zeros(shape=(h-2, w-2, self.num_filters))
        for im_region, i, j in self.iterate_regions(input):
            output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))
        return output

# Metode ini mengimplementasikan proses backpropagation untuk layer konvolusi.
#Menghitung gradien dari loss terhadap filter menggunakan chain rule, dan kemudian memperbarui filter dengan gradien tersebut.
#dL_dout = loss gradient
    def backprop(self, dL_dout, learning_rate):
        dL_dfilters = np.zeros(self.filters.shape)
        for im_region, i, j in self.iterate_regions(self.last_input):
            for f in range(self.num_filters):
                dL_dfilters[f] += dL_dout[i, j, f] * im_region


        self.filters -= learning_rate * dL_dfilters
        return None

max pool

In [ ]:
class MaxPool(object):

# Metode ini digunakan untuk mengiterasi melalui semua region 2x2
    def iterate_regions(self, image):
        h, w, _ = image.shape

        new_h = h // 2
        new_w = w // 2

        for i in range(new_h):
            for j in range(new_w):
                im_region = image[(i*2):(i*2 + 2), (j*2):(j * 2 + 2)]
                yield im_region, i, j

# melakukan proses max pooling pada input. Matriks input diiterasi melalui semua region 2x2 menggunakan iterate region
    def forward(self, input):
        self.last_input = input
        h, w, num_filters = input.shape
        output = np.zeros(shape=(h // 2, w // 2, num_filters))
        for im_region, i, j in self.iterate_regions(input):
            output[i, j] = np.amax(im_region, axis=(0, 1))
        return output

# metode ini mengimplementasikan proses backpropagation untuk layer max pooling.
# return gradien loss
    def backprop(self, dL_dout):
        dL_dinput = np.zeros(shape=self.last_input.shape)
        for im_region, i, j in self.iterate_regions(self.last_input):
            h, w, f = im_region.shape
            amax = np.amax(im_region, axis=(0, 1))
            for i2 in range(h):
                for j2 in range(w):
                    for f2 in range(f):
                        if im_region[i2, j2, f2] == amax[f2]:
                            dL_dinput[i*2 + i2, j*2 + j2,
                                      f2] = dL_dout[i, j, f2]
        return dL_dinput

soft max

In [ ]:
class Softmax:
# inisialisasi objek Softmax dengan menghasilkan bobot (weights) secara acak dan menginisialisasi bias dengan nilai nol.
    def __init__(self, input_len, nodes):
        self.weights = np.random.randn(input_len, nodes) / input_len
        self.biases = np.zeros(shape=nodes)

# roses feedforward pada layer Softmax. Input di-flatten menjadi vektor satu dimensi,
# kemudian dihitung totalnya dengan mengalikan dengan bobot (weights), menambahkan bias, dan kemudian diaplikasikan fungsi softmax.
    def forward(self, input):
        self.last_input_shape = input.shape
        input = input.flatten()
        self.last_input = input
        totals = np.dot(input, self.weights) + self.biases
        self.last_totals = totals
        exp = np.exp(totals)
        return exp / np.sum(exp, axis=0)

# mengimplementasikan proses backpropagation untuk layer Softmax.
# menghitung gradien loss terhadap input, bobot, dan bias
    def backprop(self, dL_dout, learning_rate):

        for i, gradient in enumerate(dL_dout):
            if gradient == 0:
                continue
            # e^totals
            t_exp = np.exp(self.last_totals)

            # sum dari semua e^totals
            S = np.sum(t_exp)

            # gradients output[i] dari total totals
            dout_dt = -t_exp[i] * t_exp / S**2

            # dout = turunan output class
            dout_dt[i] = t_exp[i] * (S - t_exp[i]) / S**2

            # gradien total untuk weight, bias, input
            dt_dw = self.last_input
            dt_db = 1
            dt_dinputs = self.weights

            # loss gradient untuk total
            dL_dt = gradient * dout_dt

            # loss gradient untuk weight, bias, input
            dL_dw = dt_dw[np.newaxis].T @ dL_dt[np.newaxis]
            dL_db = dL_dt * dt_db
            dL_dinputs = dt_dinputs @ dL_dt

            # update weight dan bias
            self.weights -= learning_rate * dL_dw
            self.biases -= learning_rate * dL_db

            return dL_dinputs.reshape(self.last_input_shape)


Train dataset

In [ ]:
train_images, train_labels, test_images, test_labels = getMnistData(
    reshaped=False)
train_images = train_images[:500]
train_labels = train_labels[:500]
test_images = test_images[:200]
test_labels = test_labels[:200]

conv_layer = Conv(num_filters=8)       # 28x28x1 => 26x26x8
pooling_layer = MaxPool()                # 26x26x8 => 13x13x8
softmax_layer = Softmax(13*13*8, 10)      # 13x13x8 => 10


def forward(image, label):
    # konversi image [0, 255] => [-0.5, 0.5]
    out = conv_layer.forward((image / 255) - 0.5)
    out = pooling_layer.forward(out)
    out = softmax_layer.forward(out)

    loss = -np.log(out[label])
    acc = 1 if np.argmax(out) == label else 0
    return out, loss, acc


def train(im, label, lr=.005):
    # forward
    out, loss, acc = forward(im, label)
    # gradient awal
    gradient = np.zeros(10)
    gradient[label] = -1 / out[label]
    # backprop
    gradient = softmax_layer.backprop(gradient, lr)
    gradient = pooling_layer.backprop(gradient)
    conv_layer.backprop(gradient, lr)

    return loss, acc


print('MNIST CNN from Scratch')
epochs = 5

# Training
for epoch in range(epochs):
    print(f"-------Epoch {epoch+1}-------")
    # mengacak data train
    permutation = np.random.permutation(len(train_images))
    train_images = train_images[permutation]
    train_labels = train_labels[permutation]

    # Train
    loss = 0
    num_correct = 0
    for i, (im, label) in enumerate(zip(train_images, train_labels)):
        if i > 0 and i % 200 == 99:
            print(
                f'[langkah {i}] 200 langkah terakhir: rata-rata loss {loss/100:.3f} | akurasi: {num_correct}')

            loss = 0
            num_correct = 0

        l, acc = train(im, label)
        loss += l
        num_correct += acc

MNIST CNN from Scratch
-------Epoch 1-------
[langkah 99] 200 langkah terakhir: rata-rata loss 2.248 | akurasi: 19
[langkah 299] 200 langkah terakhir: rata-rata loss 3.728 | akurasi: 89
[langkah 499] 200 langkah terakhir: rata-rata loss 1.975 | akurasi: 145
-------Epoch 2-------
[langkah 99] 200 langkah terakhir: rata-rata loss 0.681 | akurasi: 78
[langkah 299] 200 langkah terakhir: rata-rata loss 1.203 | akurasi: 161
[langkah 499] 200 langkah terakhir: rata-rata loss 1.209 | akurasi: 163
-------Epoch 3-------
[langkah 99] 200 langkah terakhir: rata-rata loss 0.503 | akurasi: 82
[langkah 299] 200 langkah terakhir: rata-rata loss 0.785 | akurasi: 178
[langkah 499] 200 langkah terakhir: rata-rata loss 0.992 | akurasi: 167
-------Epoch 4-------
[langkah 99] 200 langkah terakhir: rata-rata loss 0.298 | akurasi: 90
[langkah 299] 200 langkah terakhir: rata-rata loss 0.699 | akurasi: 178
[langkah 499] 200 langkah terakhir: rata-rata loss 0.885 | akurasi: 174
-------Epoch 5-------
[langkah 99]

Test

In [ ]:
# Testing the network
print('\n--- Test CNN ---')

loss = 0
num_correct = 0
for im, label in zip(test_images, test_labels):
    _, l, acc = forward(im, label)
    loss += l
    num_correct += acc

num_test = len(test_labels)
print(f'tes loss: {loss / num_test}')
print(f'tes akurasi: {num_correct / num_test}')



--- Test CNN ---
tes loss: 0.6155041187261642
tes akurasi: 0.78
